In [2]:
musakhan999: explain this code to me

GitHub Copilot: This code implements a **Two-Echelon Vehicle Routing Problem (2E-VRP)** solution using Google OR-Tools. Here's what it does:

## Overview
The system models a supply chain with two levels of distribution:
- **Echelon 1**: Large trucks deliver from main warehouse (Lahore) to regional satellites
- **Echelon 2**: Smaller trucks deliver from satellites to final sub-warehouses

## Key Components

### Helper Functions
```python
def euclidean_km(p1, p2) -> int:
    # Calculates straight-line distance between two coordinate points
    
def build_distance_matrix(points):
    # Creates a square matrix of distances between all locations
    
def solve_vrp(distance_matrix, demands, vehicle_capacities):
    # Uses OR-Tools to solve the Vehicle Routing Problem
```

### Data Structure
- **Cities**: Main locations with (x,y) coordinates relative to Lahore at (0,0)
- **Satellites**: Each has coordinates and sub-warehouses to serve
- **Demands**: How much cargo each sub-warehouse needs (in tons)
- **Vehicle Capacities**: Different truck sizes for each echelon

### Two-Phase Solution

**Phase 1 - Echelon 2 (Satellites → Sub-warehouses)**:
- For each satellite city, solve a VRP to serve its sub-warehouses
- Uses smaller 7-ton trucks
- Calculates total demand each satellite needs to receive

**Phase 2 - Echelon 1 (Lahore → Satellites)**:
- Uses the demand totals from Phase 1
- Solves VRP with larger 20-ton trucks from Lahore
- Delivers exactly what each satellite needs

### Example Output
The code will show:
- Route plans for each vehicle
- Total distances traveled
- Load capacities utilized
- How much cargo flows through each satellite

This approach is common in real logistics where you have regional distribution centers that need to be supplied from a main warehouse, then distribute locally to final destinations.

musakhan999: explain code in flow way of what is happening refer to the relevant code too, maybe lets do things step by step if possible and try to run so i can understand, i am already in a jupyternotebook, so there wont be any issues

GitHub Copilot: Let's break down this Two-Echelon Vehicle Routing Problem step by step and run it piece by piece so you can understand the flow!

## Step 1: Install Dependencies and Import Libraries

First, let's make sure we have the required package:

````python
!python -m pip install ortools
````

````python
from __future__ import annotations
from typing import List, Dict, Tuple
import math
from dataclasses import dataclass

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

print("Libraries imported successfully!")
````

## Step 2: Helper Functions

Let's understand the utility functions:

````python
def euclidean_km(p1: Tuple[float, float], p2: Tuple[float, float]) -> int:
    """Return rounded Euclidean distance in km (as an int for OR‑Tools)."""
    return int(round(math.hypot(p1[0]-p2[0], p1[1]-p2[1])))

# Test the distance function
point_a = (0, 0)  # Lahore
point_b = (0, 72) # Gujranwala
distance = euclidean_km(point_a, point_b)
print(f"Distance from Lahore to Gujranwala: {distance} km")
````

````python
def build_distance_matrix(points: List[Tuple[float, float]]) -> List[List[int]]:
    n = len(points)
    m = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                m[i][j] = euclidean_km(points[i], points[j])
    return m

# Test with a small example
test_points = [(0, 0), (0, 72), (-40, 10)]
test_matrix = build_distance_matrix(test_points)
print("Distance matrix:")
for i, row in enumerate(test_matrix):
    print(f"Point {i}: {row}")
````

````python
@dataclass
class VRPResult:
    routes: List[List[int]]
    route_loads: List[int]
    total_distance: int

print("VRPResult dataclass defined - this will store our routing solutions")
````

## Step 3: Define the Scenario Data

Let's set up our Pakistani cities and their coordinates:

````python
# Main warehouse and satellite cities
LAHORE = (0, 0)
CITIES = {
    "Lahore": LAHORE,
    "Gujranwala": (0, 72),      # ~70 km north
    "Sheikhupura": (-40, 10),   # ~40 km WNW
    "Kasur": (0, -55),          # ~55 km south
}

print("Main cities:")
for city, coords in CITIES.items():
    print(f"  {city}: {coords}")
````

````python
# Sub-warehouses that each satellite serves
SATELLITES = {
    "Gujranwala": {
        "coords": CITIES["Gujranwala"],
        "subs": {
            "Kamoke": (0, 52),
            "Eminabad": (-8, 66),
            "Nowshera Virkan": (-18, 80),
        },
    },
    "Sheikhupura": {
        "coords": CITIES["Sheikhupura"],
        "subs": {
            "Ferozewala": (-10, 0),
            "Muridke": (10, 40),
            "Safdarabad": (-70, 20),
        },
    },
    "Kasur": {
        "coords": CITIES["Kasur"],
        "subs": {
            "Pattoki": (10, -80),
            "Chunian": (-15, -95),
            "Kot Radha Kishan": (20, -50),
        },
    },
}

print("Satellite networks:")
for sat_name, sat_data in SATELLITES.items():
    print(f"\n{sat_name} serves:")
    for sub_name, sub_coords in sat_data["subs"].items():
        print(f"  - {sub_name}: {sub_coords}")
````

````python
# How much cargo each sub-warehouse needs (in tons)
DEMANDS_TONS = {
    # Gujranwala cluster
    "Kamoke": 4,
    "Eminabad": 3,
    "Nowshera Virkan": 5,
    # Sheikhupura cluster
    "Ferozewala": 3,
    "Muridke": 5,
    "Safdarabad": 4,
    # Kasur cluster
    "Pattoki": 6,
    "Chunian": 5,
    "Kot Radha Kishan": 3,
}

print("Cargo demands (tons):")
total_demand = 0
for location, demand in DEMANDS_TONS.items():
    print(f"  {location}: {demand}t")
    total_demand += demand
print(f"Total system demand: {total_demand}t")
````

````python
# Vehicle capacities for each echelon
E1_CAPACITIES = [20, 20]  # two 20‑ton trucks for Lahore → satellites
E2_CAPACITIES = {
    "Gujranwala": [7, 7],    # two 7-ton trucks per satellite
    "Sheikhupura": [7, 7],
    "Kasur": [7, 7],
}

print("Vehicle fleets:")
print(f"Echelon 1 (Lahore → satellites): {len(E1_CAPACITIES)} trucks of {E1_CAPACITIES[0]}t each")
print("Echelon 2 (satellites → sub-warehouses):")
for sat, caps in E2_CAPACITIES.items():
    print(f"  {sat}: {len(caps)} trucks of {caps[0]}t each")
````

Now let's run a small example to understand what happens next! Would you like me to continue with the VRP solver function and then show how we solve each echelon?

SyntaxError: invalid decimal literal (117739345.py, line 3)

In [ ]:
from __future__ import annotations
from typing import List, Dict, Tuple
import math
from dataclasses import dataclass

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
LAHORE = (0, 0)
CITIES = {
    "Lahore": LAHORE,
    "Gujranwala": (0, 72),      # ~70 km north
    "Sheikhupura": (-40, 10),   # ~40 km WNW
    "Kasur": (0, -55),          # ~55 km south
}

print("Main cities:")
for city, coords in CITIES.items():
    print(f"  {city}: {coords}")

Main cities:
  Lahore: (0, 0)
  Gujranwala: (0, 72)
  Sheikhupura: (-40, 10)
  Kasur: (0, -55)


In [ ]:
# Let's visualize this:
print("City positions relative to Lahore:")
print("Lahore:      (0, 0)   - Origin point")
print("Gujranwala:  (0, 72)  - 72km North, same longitude")
print("Sheikhupura: (-40, 10) - 40km West, 10km North") 
print("Kasur:       (0, -55) - 55km South, same longitude")

# Test the distance calculation
def euclidean_km(p1, p2):
    return int(round(math.hypot(p1[0]-p2[0], p1[1]-p2[1])))

print("\nActual distances from Lahore:")
for city, coords in CITIES.items():
    if city != "Lahore":
        distance = euclidean_km(LAHORE, coords)
        print(f"  {city}: {distance} km")

City positions relative to Lahore:
Lahore:      (0, 0)   - Origin point
Gujranwala:  (0, 72)  - 72km North, same longitude
Sheikhupura: (-40, 10) - 40km West, 10km North
Kasur:       (0, -55) - 55km South, same longitude

Actual distances from Lahore:
  Gujranwala: 72 km
  Sheikhupura: 41 km
  Kasur: 55 km


In [ ]:
SATELLITES = {
    "Gujranwala": {
        "coords": CITIES["Gujranwala"],
        "subs": {
            "Kamoke": (0, 52),
            "Eminabad": (-8, 66),
            "Nowshera Virkan": (-18, 80),
        },
    },
    "Sheikhupura": {
        "coords": CITIES["Sheikhupura"],
        "subs": {
            "Ferozewala": (-10, 0),
            "Muridke": (10, 40),
            "Safdarabad": (-70, 20),
        },
    },
    "Kasur": {
        "coords": CITIES["Kasur"],
        "subs": {
            "Pattoki": (10, -80),
            "Chunian": (-15, -95),
            "Kot Radha Kishan": (20, -50),
        },
    },
}

print("Satellite networks:")
for sat_name, sat_data in SATELLITES.items():
    print(f"\n{sat_name} serves:")
    for sub_name, sub_coords in sat_data["subs"].items():
        print(f"  - {sub_name}: {sub_coords}")

Satellite networks:

Gujranwala serves:
  - Kamoke: (0, 52)
  - Eminabad: (-8, 66)
  - Nowshera Virkan: (-18, 80)

Sheikhupura serves:
  - Ferozewala: (-10, 0)
  - Muridke: (10, 40)
  - Safdarabad: (-70, 20)

Kasur serves:
  - Pattoki: (10, -80)
  - Chunian: (-15, -95)
  - Kot Radha Kishan: (20, -50)


In [ ]:
# Fertilizer bag example - let's compare both approaches:

# Method 1: Using BAGS
FERTILIZER_BAG_WEIGHT = 50  # kg per bag
DEMANDS_BAGS = {
    # Gujranwala cluster
    "Kamoke": 80,        # 80 bags = 4 tons
    "Eminabad": 60,      # 60 bags = 3 tons  
    "Nowshera Virkan": 100,  # 100 bags = 5 tons
    # Sheikhupura cluster
    "Ferozewala": 60,    # 60 bags = 3 tons
    "Muridke": 100,      # 100 bags = 5 tons
    "Safdarabad": 80,    # 80 bags = 4 tons
    # Kasur cluster
    "Pattoki": 120,      # 120 bags = 6 tons
    "Chunian": 100,      # 100 bags = 5 tons
    "Kot Radha Kishan": 60,  # 60 bags = 3 tons
}

# Method 2: Using TONS (current approach)
DEMANDS_TONS = {
    "Kamoke": 4,
    "Eminabad": 3,
    "Nowshera Virkan": 5,
    "Ferozewala": 3,
    "Muridke": 5,
    "Safdarabad": 4,
    "Pattoki": 6,
    "Chunian": 5,
    "Kot Radha Kishan": 3,
}

print("Comparison:")
print("Method 1 - Bags: Total =", sum(DEMANDS_BAGS.values()), "bags")
print("Method 2 - Tons: Total =", sum(DEMANDS_TONS.values()), "tons")
print("Weight equivalence:", sum(DEMANDS_BAGS.values()) * FERTILIZER_BAG_WEIGHT / 1000, "tons")

Comparison:
Method 1 - Bags: Total = 760 bags
Method 2 - Tons: Total = 38 tons
Weight equivalence: 38.0 tons


In [ ]:
# How much cargo each sub-warehouse needs (in tons)
DEMANDS_TONS = {
    # Gujranwala cluster
    "Kamoke": 4,
    "Eminabad": 3,
    "Nowshera Virkan": 5,
    # Sheikhupura cluster
    "Ferozewala": 3,
    "Muridke": 5,
    "Safdarabad": 4,
    # Kasur cluster
    "Pattoki": 6,
    "Chunian": 5,
    "Kot Radha Kishan": 3,
}

print("Cargo demands (tons):")
total_demand = 0
for location, demand in DEMANDS_TONS.items():
    print(f"  {location}: {demand}t")
    total_demand += demand
print(f"Total system demand: {total_demand}t")

Cargo demands (tons):
  Kamoke: 4t
  Eminabad: 3t
  Nowshera Virkan: 5t
  Ferozewala: 3t
  Muridke: 5t
  Safdarabad: 4t
  Pattoki: 6t
  Chunian: 5t
  Kot Radha Kishan: 3t
Total system demand: 38t


In [ ]:
# Vehicle capacities for fertilizer distribution
E1_CAPACITIES = [15, 15]  # Two 15-ton trucks for main routes
E2_CAPACITIES = {
    "Gujranwala": [5, 5],    # Two 5-ton trucks per satellite
    "Sheikhupura": [5, 5],   # (smaller trucks for local delivery)
    "Kasur": [5, 5],
}

print("Vehicle fleets:")
print(f"Echelon 1 (Lahore → satellites): {len(E1_CAPACITIES)} trucks of {E1_CAPACITIES[0]}t each")
print("Echelon 2 (satellites → sub-warehouses):")
for sat, caps in E2_CAPACITIES.items():
    print(f"  {sat}: {len(caps)} trucks of {caps[0]}t each")


Vehicle fleets:
Echelon 1 (Lahore → satellites): 2 trucks of 15t each
Echelon 2 (satellites → sub-warehouses):
  Gujranwala: 2 trucks of 5t each
  Sheikhupura: 2 trucks of 5t each
  Kasur: 2 trucks of 5t each


Fertilizer delivery fleet:
Main warehouse → Satellites: 2 trucks of 15t each
Satellite → Sub-districts:
  Gujranwala: 2 trucks of 5t each
  Sheikhupura: 2 trucks of 5t each
  Kasur: 2 trucks of 5t each

For 50kg bags: 1 ton = 20 bags


### Rethinking

In [ ]:
import osmnx as ox
import folium
import geopandas as gpd
import numpy as np
from folium import plugins
import networkx as nx
# vpr_sample_pakistan.py
# Requirements: pip install ortools



print("✅ OSMnx and mapping libraries imported successfully!")

✅ OSMnx and mapping libraries imported successfully!


In [2]:
from math import radians, sin, cos, sqrt, atan2

In [3]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# ----- Sample coordinates (Lahore-ish) -----
# Warehouse first, then shops (lat, lon)
locations = [
    ("Warehouse", 31.5204, 74.3587),  # central Lahore (example)
    ("Shop A",    31.5340, 74.3572),
    ("Shop B",    31.5157, 74.3673),
    ("Shop C",    31.5280, 74.3810),
    ("Shop D",    31.5060, 74.3430),
    ("Shop E",    31.5400, 74.3440),
    ("Shop F",    31.5220, 74.3320),
    ("Shop G",    31.5100, 74.3790),
    ("Shop H",    31.5380, 74.3720),
]

# Demands in bags (warehouse has 0)
demands = [0, 40, 20, 30, 10, 12, 15, 18, 20]

# Vehicle fleet: (count, capacity in bags, cost_per_km)
# Example heterogenous fleet: 2 trucks and 2 vans
vehicles = [
    {"count": 1, "capacity": 200, "cost_per_km": 1.0},  # big truck: 1.0 currency/km
    {"count": 2, "capacity": 80,  "cost_per_km": 0.7},  # medium van
    {"count": 1, "capacity": 50,  "cost_per_km": 0.5},  # small van
]

# Helper: haversine distance (km)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Build distance matrix (approx road distance = haversine * factor)
n = len(locations)
dist_km = [[0]*n for _ in range(n)]
road_factor = 1.25  # scale to approximate road path vs straight-line
for i in range(n):
    for j in range(n):
        if i == j:
            dist_km[i][j] = 0
        else:
            dist_km[i][j] = haversine_km(locations[i][1], locations[i][2],
                                         locations[j][1], locations[j][2]) * road_factor

# Convert to integer costs (meters or scaled integer)
SCALE = 100  # so 1 km = 100 units (keeps integers but preserves decimals)
dist_matrix = [[int(round(d * SCALE)) for d in row] for row in dist_km]

# Flatten vehicles into list of vehicle specs
vehicle_specs = []
for v in vehicles:
    for _ in range(v["count"]):
        vehicle_specs.append((v["capacity"], v["cost_per_km"]))
num_vehicles = len(vehicle_specs)
depot = 0

# OR-Tools: create routing model
manager = pywrapcp.RoutingIndexManager(len(dist_matrix), num_vehicles, depot)
routing = pywrapcp.RoutingModel(manager)

# For heterogeneous per-vehicle cost, we register a separate evaluator for each vehicle.
def make_distance_callback(matrix):
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return matrix[from_node][to_node]
    return distance_callback

# Register base evaluator (we'll set per-vehicle later)
base_callback = make_distance_callback(dist_matrix)
base_evaluator_idx = routing.RegisterTransitCallback(base_callback)

# Set arc cost evaluator per vehicle with per-km multiplier baked in
for vid, (cap, cost_per_km) in enumerate(vehicle_specs):
    # We need an evaluator that multiplies base distance by vehicle-specific cost multiplier.
    # To do that, create a wrapper callback that multiplies by (cost_per_km * SCALE)
    # Since base is in SCALE units of km, cost per unit = cost_per_km / SCALE_km? Simpler:
    multiplier = cost_per_km  # currency per km
    def make_eval(m):
        def eval_fn(from_index, to_index):
            f = manager.IndexToNode(from_index)
            t = manager.IndexToNode(to_index)
            # distance in km = dist_matrix[f][t] / SCALE
            # cost = (distance_km) * m; we convert to integer by multiply by 100 and round
            return int(round(dist_matrix[f][t] / SCALE * m * 100))  # integer cost units (e.g., paisa)
        return eval_fn
    evaluator_index = routing.RegisterTransitCallback(make_eval(multiplier))
    routing.SetArcCostEvaluatorOfVehicle(evaluator_index, vid)

# Add capacity dimension
def demand_callback(from_index):
    node = manager.IndexToNode(from_index)
    return demands[node]
demand_cb_idx = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_cb_idx,
    0,  # null capacity slack
    [spec[0] for spec in vehicle_specs],  # vehicle capacities
    True,
    "Capacity"
)

# Optional: minimize number of vehicles used via small fixed cost
# OR-Tools provides SetFixedCostOfVehicle in newer versions; if not available, you can add a penalty to unused vehicles.
for v in range(num_vehicles):
    routing.SetFixedCostOfVehicle(1000, v)  # a fixed cost in same integer units (tweak as needed)

# Solve
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.time_limit.seconds = 10  # small for prototype
search_parameters.log_search = False

solution = routing.SolveWithParameters(search_parameters)
if solution:
    total_cost = 0
    print("Solution routes:")
    for v in range(num_vehicles):
        index = routing.Start(v)
        route = []
        load = 0
        route_cost = 0
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(locations[node][0])
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            route_cost += routing.GetArcCostForVehicle(index, next_index, v)
            index = next_index
        route.append(locations[manager.IndexToNode(index)][0])  # depot at end
        if len(route) > 2:  # vehicle used to visit something
            # route_cost is in integer cost units (we multiplied by 100)
            print(f"Vehicle {v}: capacity={vehicle_specs[v][0]} cost_per_km={vehicle_specs[v][1]}")
            print("  -> " + " -> ".join(route))
            print(f"  Load: {load} bags")
            print(f"  Estimated cost units: {route_cost}") 
            total_cost += route_cost
    print(f"Total estimated cost units: {total_cost}")
    # Convert cost units back to currency-ish (we multiplied by 100 above)
    print(f"Estimated total cost (currency units): {total_cost/100:.2f}")
else:
    print("No solution found.")


Solution routes:
Vehicle 1: capacity=80 cost_per_km=0.7
  -> Warehouse -> Shop B -> Shop G -> Warehouse
  Load: 38 bags
  Estimated cost units: 1394
Vehicle 2: capacity=80 cost_per_km=0.7
  -> Warehouse -> Shop A -> Shop E -> Shop F -> Shop D -> Warehouse
  Load: 77 bags
  Estimated cost units: 1831
Vehicle 3: capacity=50 cost_per_km=0.5
  -> Warehouse -> Shop C -> Shop H -> Warehouse
  Load: 50 bags
  Estimated cost units: 1376
Total estimated cost units: 4601
Estimated total cost (currency units): 46.01


#### Validation

In [4]:
demands

[0, 40, 20, 30, 10, 12, 15, 18, 20]

In [5]:
sum(demands)

165

In [6]:
vehicles

[{'count': 1, 'capacity': 200, 'cost_per_km': 1.0},
 {'count': 2, 'capacity': 80, 'cost_per_km': 0.7},
 {'count': 1, 'capacity': 50, 'cost_per_km': 0.5}]

In [4]:
# Step 1: Let's first understand our setup
print("=== SINGLE-ECHELON VRP: One Warehouse → Multiple Shops ===\n")

# Our locations (warehouse + shops)
locations = [
    ("Warehouse", 31.5204, 74.3587),  # central Lahore (example)
    ("Shop A",    31.5340, 74.3572),
    ("Shop B",    31.5157, 74.3673),
    ("Shop C",    31.5280, 74.3810),
    ("Shop D",    31.5060, 74.3430),
    ("Shop E",    31.5400, 74.3440),
    ("Shop F",    31.5220, 74.3320),
    ("Shop G",    31.5100, 74.3790),
    ("Shop H",    31.5380, 74.3720),
]

print("📍 Locations:")
for i, (name, lat, lon) in enumerate(locations):
    print(f"  {i}: {name} at ({lat:.4f}, {lon:.4f})")

=== SINGLE-ECHELON VRP: One Warehouse → Multiple Shops ===

📍 Locations:
  0: Warehouse at (31.5204, 74.3587)
  1: Shop A at (31.5340, 74.3572)
  2: Shop B at (31.5157, 74.3673)
  3: Shop C at (31.5280, 74.3810)
  4: Shop D at (31.5060, 74.3430)
  5: Shop E at (31.5400, 74.3440)
  6: Shop F at (31.5220, 74.3320)
  7: Shop G at (31.5100, 74.3790)
  8: Shop H at (31.5380, 74.3720)


In [5]:
# Step 2: Demands - how much each location needs
demands = [0, 40, 20, 30, 10, 12, 15, 18, 20]  # warehouse needs 0, shops need fertilizer bags

print("\n📦 Demands (fertilizer bags):")
for i, (name, _, _) in enumerate(locations):
    print(f"  {name}: {demands[i]} bags")

total_demand = sum(demands)
print(f"\n📊 Total demand: {total_demand} bags")


📦 Demands (fertilizer bags):
  Warehouse: 0 bags
  Shop A: 40 bags
  Shop B: 20 bags
  Shop C: 30 bags
  Shop D: 10 bags
  Shop E: 12 bags
  Shop F: 15 bags
  Shop G: 18 bags
  Shop H: 20 bags

📊 Total demand: 165 bags


In [6]:
# Step 3: Our vehicle fleet - different truck types with different costs
vehicles = [
    {"count": 1, "capacity": 200, "cost_per_km": 100.0},  # 1 big truck: expensive but high capacity
    {"count": 2, "capacity": 80,  "cost_per_km": 70},  # 2 medium vans: balanced
    {"count": 1, "capacity": 50,  "cost_per_km": 50},  # 1 small van: cheap but low capacity
]

print("\n🚛 Available vehicles:")
total_vehicles = 0
total_capacity = 0
for v in vehicles:
    print(f"  {v['count']}x {v['capacity']}-bag truck(s) @ ₨{v['cost_per_km']}/km")
    total_vehicles += v['count']
    total_capacity += v['count'] * v['capacity']

print(f"\n📋 Fleet summary:")
print(f"  Total vehicles: {total_vehicles}")
print(f"  Total capacity: {total_capacity} bags")
print(f"  Demand vs Capacity: {total_demand}/{total_capacity} bags ({total_demand/total_capacity*100:.1f}% utilization)")


🚛 Available vehicles:
  1x 200-bag truck(s) @ ₨100.0/km
  2x 80-bag truck(s) @ ₨70/km
  1x 50-bag truck(s) @ ₨50/km

📋 Fleet summary:
  Total vehicles: 4
  Total capacity: 410 bags
  Demand vs Capacity: 165/410 bags (40.2% utilization)


In [10]:
# Step 4: Calculate distances between all locations using GPS coordinates
from math import radians, sin, cos, sqrt, atan2

def haversine_km(lat1, lon1, lat2, lon2):
    """Calculate real-world distance between two GPS points"""
    R = 6371.0  # Earth's radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Build distance matrix
n = len(locations)
dist_km = [[0]*n for _ in range(n)]
road_factor = 1.25  # roads are ~25% longer than straight-line distance

print("\n📏 Distance Matrix (km):")
print("     ", end="")
for i in range(n):
    print(f"{i:5}", end="")
print()

for i in range(n):
    print(f"{i:2}: ", end="")
    for j in range(n):
        if i == j:
            dist_km[i][j] = 0
            print(f"{0:5.1f}", end="")
        else:
            dist_km[i][j] = haversine_km(locations[i][1], locations[i][2],
                                         locations[j][1], locations[j][2]) * road_factor
            print(f"{dist_km[i][j]:5.1f}", end="")
    print(f"  ({locations[i][0]})")


📏 Distance Matrix (km):
         0    1    2    3    4    5    6    7    8
 0:   0.0  1.9  1.2  2.8  2.7  3.2  3.2  2.8  2.9  (Warehouse)
 1:   1.9  0.0  2.8  2.9  4.2  1.8  3.4  4.2  1.8  (Shop A)
 2:   1.2  2.8  0.0  2.4  3.2  4.4  4.3  1.6  3.1  (Shop B)
 3:   2.8  2.9  2.4  0.0  5.4  4.7  5.9  2.5  1.8  (Shop C)
 4:   2.7  4.2  3.2  5.4  0.0  4.7  2.6  4.3  5.6  (Shop D)
 5:   3.2  1.8  4.4  4.7  4.7  0.0  2.9  5.9  3.3  (Shop E)
 6:   3.2  3.4  4.3  5.9  2.6  2.9  0.0  5.8  5.2  (Shop F)
 7:   2.8  4.2  1.6  2.5  4.3  5.9  5.8  0.0  4.0  (Shop G)
 8:   2.9  1.8  3.1  1.8  5.6  3.3  5.2  4.0  0.0  (Shop H)


In [11]:
# Step 5: Prepare the data for OR-Tools optimization
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Convert distances to integer (OR-Tools requirement)
SCALE = 100  # 1 km = 100 units for precision
dist_matrix = [[int(round(d * SCALE)) for d in row] for row in dist_km]

# Flatten vehicle specs into individual vehicles
vehicle_specs = []
for v in vehicles:
    for _ in range(v["count"]):
        vehicle_specs.append((v["capacity"], v["cost_per_km"]))

print(f"\n🔧 OR-Tools setup:")
print(f"  Distance matrix: {n}x{n} (scaled by {SCALE})")
print(f"  Vehicle specs: {len(vehicle_specs)} individual vehicles")
for i, (cap, cost) in enumerate(vehicle_specs):
    print(f"    Vehicle {i}: {cap} bags capacity, ₨{cost}/km")


🔧 OR-Tools setup:
  Distance matrix: 9x9 (scaled by 100)
  Vehicle specs: 4 individual vehicles
    Vehicle 0: 200 bags capacity, ₨100.0/km
    Vehicle 1: 80 bags capacity, ₨70/km
    Vehicle 2: 80 bags capacity, ₨70/km
    Vehicle 3: 50 bags capacity, ₨50/km


In [12]:
# Step 6: Run the VRP optimization
manager = pywrapcp.RoutingIndexManager(len(dist_matrix), len(vehicle_specs), 0)  # depot = 0 (warehouse)
routing = pywrapcp.RoutingModel(manager)

# Distance callback
def make_distance_callback(matrix):
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return matrix[from_node][to_node]
    return distance_callback

base_callback = make_distance_callback(dist_matrix)

# Set different costs per vehicle type
for vid, (cap, cost_per_km) in enumerate(vehicle_specs):
    def make_cost_eval(multiplier):
        def cost_eval(from_index, to_index):
            f = manager.IndexToNode(from_index)
            t = manager.IndexToNode(to_index)
            # Convert: distance_units/SCALE * cost_per_km * 100 (for integer precision)
            return int(round(dist_matrix[f][t] / SCALE * multiplier * 100))
        return cost_eval
    
    cost_evaluator = routing.RegisterTransitCallback(make_cost_eval(cost_per_km))
    routing.SetArcCostEvaluatorOfVehicle(cost_evaluator, vid)

# Add capacity constraints
def demand_callback(from_index):
    node = manager.IndexToNode(from_index)
    return demands[node]

demand_cb_idx = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_cb_idx, 0, [spec[0] for spec in vehicle_specs], True, "Capacity")

# Add fixed cost to minimize number of vehicles used
for v in range(len(vehicle_specs)):
    routing.SetFixedCostOfVehicle(1000, v)

print("✅ VRP model created successfully!")

✅ VRP model created successfully!


In [13]:
# Step 7: Solve and display results
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.time_limit.seconds = 10

print("\n🔍 Solving VRP...")
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print("\n✅ Solution found!")
    print("\n🚛 OPTIMIZED DELIVERY ROUTES:")
    print("=" * 50)
    
    total_cost = 0
    total_distance = 0
    vehicles_used = 0
    
    for v in range(len(vehicle_specs)):
        index = routing.Start(v)
        route = []
        load = 0
        route_cost = 0
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(locations[node][0])
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            route_cost += routing.GetArcCostForVehicle(index, next_index, v)
            route_distance += dist_km[node][manager.IndexToNode(next_index)]
            index = next_index
        
        route.append(locations[manager.IndexToNode(index)][0])  # return to warehouse
        
        if len(route) > 2:  # vehicle was actually used
            vehicles_used += 1
            capacity, cost_per_km = vehicle_specs[v]
            print(f"\n🚚 Vehicle {v+1} ({capacity}-bag truck, ₨{cost_per_km}/km):")
            print(f"   Route: {' → '.join(route)}")
            print(f"   Load: {load}/{capacity} bags ({load/capacity*100:.1f}% full)")
            print(f"   Distance: {route_distance:.1f} km")
            print(f"   Cost: ₨{route_cost/100:.2f}")
            
            total_cost += route_cost
            total_distance += route_distance
    
    print(f"\n📊 SUMMARY:")
    print(f"   Vehicles used: {vehicles_used}/{len(vehicle_specs)}")
    print(f"   Total distance: {total_distance:.1f} km")
    print(f"   Total cost: ₨{total_cost/100:.2f}")
    print(f"   Cost per bag: ₨{total_cost/100/total_demand:.2f}")
    print(f"   Avg cost per km: ₨{total_cost/100/total_distance:.2f}")
    
else:
    print("❌ No solution found!")


🔍 Solving VRP...

✅ Solution found!

🚛 OPTIMIZED DELIVERY ROUTES:

🚚 Vehicle 2 (80-bag truck, ₨70/km):
   Route: Warehouse → Shop B → Shop G → Warehouse
   Load: 38/80 bags (47.5% full)
   Distance: 5.6 km
   Cost: ₨403.40

🚚 Vehicle 3 (80-bag truck, ₨70/km):
   Route: Warehouse → Shop A → Shop E → Shop F → Shop D → Warehouse
   Load: 77/80 bags (96.2% full)
   Distance: 11.9 km
   Cost: ₨840.20

🚚 Vehicle 4 (50-bag truck, ₨50/km):
   Route: Warehouse → Shop C → Shop H → Warehouse
   Load: 50/50 bags (100.0% full)
   Distance: 7.5 km
   Cost: ₨385.50

📊 SUMMARY:
   Vehicles used: 3/4
   Total distance: 25.0 km
   Total cost: ₨1629.10
   Cost per bag: ₨9.87
   Avg cost per km: ₨65.22


In [14]:
# Install required packages




### Enchanced road calculation

In [8]:
# Install required packages
#!pip install osmnx folium geopandas


# import folium
# import geopandas as gpd
# import numpy as np
# from folium import plugins
# import networkx as nx

print("✅ OSMnx and mapping libraries imported successfully!")

✅ OSMnx and mapping libraries imported successfully!


In [9]:
import osmnx as ox

In [10]:
dist_km = [[0]*n for _ in range(n)]

NameError: name 'n' is not defined

In [11]:
# Enhanced distance calculation using real road networks
def get_road_network(locations, buffer_distance=50000):
    """Download road network around all locations"""
    # Get bounding box for all locations
    lats = [loc[1] for loc in locations]
    lons = [loc[2] for loc in locations]
    
    north = max(lats) + 0.01
    south = min(lats) - 0.01
    east = max(lons) + 0.01
    west = min(lons) - 0.01
    
    print(f"📍 Downloading road network for area: {north:.3f}, {south:.3f}, {east:.3f}, {west:.3f}")
    
    # Download road network
    G = ox.graph_from_bbox(north, south, east, west, network_type='drive')
    print(f"✅ Downloaded road network with {len(G.nodes)} nodes and {len(G.edges)} edges")
    
    return G

def calculate_real_distances(locations, graph):
    """Calculate actual road distances using OSMnx"""
    n = len(locations)
    distance_matrix = np.zeros((n, n))
    
    # Get nearest nodes for each location
    nearest_nodes = []
    for name, lat, lon in locations:
        try:
            node = ox.nearest_nodes(graph, lon, lat)
            nearest_nodes.append(node)
            print(f"📍 {name}: Found nearest road node {node}")
        except Exception as e:
            print(f"❌ Error finding node for {name}: {e}")
            nearest_nodes.append(None)
    
    # Calculate shortest paths between all pairs
    print("\n🛣️ Calculating real road distances...")
    for i in range(n):
        for j in range(n):
            if i == j:
                distance_matrix[i][j] = 0
            elif nearest_nodes[i] and nearest_nodes[j]:
                try:
                    # Get shortest path
                    path = nx.shortest_path(graph, nearest_nodes[i], nearest_nodes[j], weight='length')
                    # Calculate path length
                    path_length = sum(ox.utils_graph.get_route_edge_attributes(graph, path, 'length'))
                    distance_matrix[i][j] = path_length / 1000  # Convert to km
                except:
                    # Fallback to haversine if no path found
                    distance_matrix[i][j] = haversine_km(locations[i][1], locations[i][2], 
                                                       locations[j][1], locations[j][2])
    
    return distance_matrix, nearest_nodes

# Download road network and calculate real distances
try:
    G = get_road_network(locations)
    real_dist_km, location_nodes = calculate_real_distances(locations, G)
    
    print("\n📊 Real Road Distance Matrix (km):")
    print("     ", end="")
    for i in range(len(locations)):
        print(f"{i:6}", end="")
    print()
    
    for i in range(len(locations)):
        print(f"{i:2}: ", end="")
        for j in range(len(locations)):
            print(f"{real_dist_km[i][j]:6.1f}", end="")
        print(f"  ({locations[i][0]})")
        
except Exception as e:
    print(f"❌ Error with OSMnx: {e}")
    print("🔄 Falling back to haversine distances...")
    real_dist_km = dist_km
    G = None
    location_nodes = None

📍 Downloading road network for area: 31.550, 31.496, 74.391, 74.322
❌ Error with OSMnx: graph_from_bbox() takes 1 positional argument but 4 positional arguments (and 1 keyword-only argument) were given
🔄 Falling back to haversine distances...


NameError: name 'dist_km' is not defined

In [12]:
# First, let's see if your OSMnx network download worked
print("Current locations:")
for i, (name, lat, lon) in enumerate(locations):
    print(f"  {i}: {name} at ({lat:.4f}, {lon:.4f})")

print(f"\nBounding box for road network:")
lats = [loc[1] for loc in locations]
lons = [loc[2] for loc in locations]
print(f"  North: {max(lats) + 0.01:.4f}")
print(f"  South: {min(lats) - 0.01:.4f}")
print(f"  East: {max(lons) + 0.01:.4f}")
print(f"  West: {min(lons) - 0.01:.4f}")

Current locations:
  0: Warehouse at (31.5204, 74.3587)
  1: Shop A at (31.5340, 74.3572)
  2: Shop B at (31.5157, 74.3673)
  3: Shop C at (31.5280, 74.3810)
  4: Shop D at (31.5060, 74.3430)
  5: Shop E at (31.5400, 74.3440)
  6: Shop F at (31.5220, 74.3320)
  7: Shop G at (31.5100, 74.3790)
  8: Shop H at (31.5380, 74.3720)

Bounding box for road network:
  North: 31.5500
  South: 31.4960
  East: 74.3910
  West: 74.3220


In [13]:
# Let's try downloading the network with error handling
try:
    print("🌐 Downloading Lahore road network...")
    G = get_road_network(locations)
    print(f"✅ Success! Network has {len(G.nodes)} nodes and {len(G.edges)} edges")
    network_success = True
except Exception as e:
    print(f"❌ Network download failed: {e}")
    print("This might be due to internet connection or OSMnx server issues")
    network_success = False

🌐 Downloading Lahore road network...
📍 Downloading road network for area: 31.550, 31.496, 74.391, 74.322
❌ Network download failed: graph_from_bbox() takes 1 positional argument but 4 positional arguments (and 1 keyword-only argument) were given
This might be due to internet connection or OSMnx server issues


In [14]:
# If network download worked, calculate real distances
if network_success:
    try:
        print("🛣️ Calculating real road distances...")
        real_dist_km, location_nodes = calculate_real_distances(locations, G)
        print("✅ Real distances calculated!")
        
        # Show comparison
        print("\n📊 Distance Comparison (km):")
        print("Location pair | Haversine | Real Road | Difference")
        print("-" * 50)
        for i in range(len(locations)):
            for j in range(i+1, len(locations)):
                haversine_dist = dist_km[i][j]
                real_dist = real_dist_km[i][j]
                diff = ((real_dist - haversine_dist) / haversine_dist) * 100
                print(f"{locations[i][0][:10]} → {locations[j][0][:10]} | {haversine_dist:8.1f} | {real_dist:8.1f} | {diff:+6.1f}%")
                
    except Exception as e:
        print(f"❌ Distance calculation failed: {e}")
        real_dist_km = dist_km


In [15]:
# Try downloading a smaller area
try:
    # Smaller bounding box
    center_lat, center_lon = 31.5204, 74.3587  # Your warehouse location
    G = ox.graph_from_point((center_lat, center_lon), dist=2000, network_type='drive')
    print("✅ Smaller area download worked!")
except:
    print("❌ Still failing - let's use haversine distances")

✅ Smaller area download worked!


In [16]:
# Simple test to see if OSMnx works
try:
    print("🧪 Testing OSMnx with a small area around Lahore...")
    
    # Small test area
    north, south, east, west = 31.53, 31.51, 74.37, 74.35
    
    print(f"📍 Downloading small test area: {north}, {south}, {east}, {west}")
    test_graph = ox.graph_from_bbox(north, south, east, west, network_type='drive')
    
    print(f"✅ SUCCESS! Downloaded {len(test_graph.nodes)} nodes and {len(test_graph.edges)} edges")
    print("🌐 OSMnx is working - it can automatically download maps!")
    
except Exception as e:
    print(f"❌ OSMnx failed: {e}")
    print("💡 This could be due to:")
    print("   - Internet connection issues")
    print("   - OpenStreetMap server problems") 
    print("   - Firewall blocking the request")

🧪 Testing OSMnx with a small area around Lahore...
📍 Downloading small test area: 31.53, 31.51, 74.37, 74.35
❌ OSMnx failed: graph_from_bbox() takes 1 positional argument but 4 positional arguments (and 1 keyword-only argument) were given
💡 This could be due to:
   - Internet connection issues
   - OpenStreetMap server problems
   - Firewall blocking the request


In [17]:
# Complete OSMnx-based VRP solution
import osmnx as ox
import folium
import numpy as np
import networkx as nx
from math import radians, sin, cos, sqrt, atan2
#

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [18]:
#from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [19]:
# Our delivery locations in Lahore
locations = [
    ("Warehouse", 31.5204, 74.3587),  # central Lahore (depot)
    ("Shop A",    31.5340, 74.3572),
    ("Shop B",    31.5157, 74.3673),
    ("Shop C",    31.5280, 74.3810),
    ("Shop D",    31.5060, 74.3430),
    ("Shop E",    31.5400, 74.3440),
    ("Shop F",    31.5220, 74.3320),
    ("Shop G",    31.5100, 74.3790),
    ("Shop H",    31.5380, 74.3720),
]

# Demands (warehouse = 0, shops need fertilizer bags)
demands = [0, 40, 20, 30, 10, 12, 15, 18, 20]

# Vehicle fleet
vehicles = [
    {"count": 1, "capacity": 200, "cost_per_km": 100.0},  # big truck
    {"count": 2, "capacity": 80,  "cost_per_km": 70.0},   # medium vans
    {"count": 1, "capacity": 50,  "cost_per_km": 50.0},   # small van
]

print("📍 Setup complete:")
print(f"  Locations: {len(locations)}")
print(f"  Total demand: {sum(demands)} bags")
print(f"  Vehicle types: {len(vehicles)}")

📍 Setup complete:
  Locations: 9
  Total demand: 165 bags
  Vehicle types: 3


In [20]:
# Download road network using the method that worked
def download_road_network_safe(locations):
    """Download road network using smaller area method"""
    try:
        # Use center point method (which worked for you)
        center_lat = sum(loc[1] for loc in locations) / len(locations)
        center_lon = sum(loc[2] for loc in locations) / len(locations)
        
        print(f"📍 Downloading network around center: ({center_lat:.4f}, {center_lon:.4f})")
        
        # Download with 5km radius (adjust if needed)
        G = ox.graph_from_point((center_lat, center_lon), dist=5000, network_type='drive')
        
        print(f"✅ SUCCESS! Network downloaded:")
        print(f"   Nodes: {len(G.nodes)}")
        print(f"   Edges: {len(G.edges)}")
        
        return G, True
        
    except Exception as e:
        print(f"❌ Network download failed: {e}")
        return None, False

# Download the network
G, network_success = download_road_network_safe(locations)

📍 Downloading network around center: (31.5238, 74.3594)
✅ SUCCESS! Network downloaded:
   Nodes: 16351
   Edges: 41697


In [21]:
def haversine_km(lat1, lon1, lat2, lon2):
    """Fallback distance calculation"""
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

def calculate_distance_matrix(locations, graph=None):
    """Calculate distance matrix using OSMnx or fallback to haversine"""
    n = len(locations)
    dist_matrix = np.zeros((n, n))
    
    if graph is not None:
        print("🛣️ Calculating real road distances...")
        
        # Find nearest nodes for each location
        nearest_nodes = []
        for i, (name, lat, lon) in enumerate(locations):
            try:
                node = ox.nearest_nodes(graph, lon, lat)
                nearest_nodes.append(node)
                print(f"   {name}: found node {node}")
            except Exception as e:
                print(f"   ❌ {name}: error - {e}")
                nearest_nodes.append(None)
        
        # Calculate distances
        for i in range(n):
            for j in range(n):
                if i == j:
                    dist_matrix[i][j] = 0
                elif nearest_nodes[i] and nearest_nodes[j]:
                    try:
                        # Real road distance
                        path = nx.shortest_path(graph, nearest_nodes[i], nearest_nodes[j], weight='length')
                        path_length = sum(ox.utils_graph.get_route_edge_attributes(graph, path, 'length'))
                        dist_matrix[i][j] = path_length / 1000  # Convert to km
                    except:
                        # Fallback to haversine
                        dist_matrix[i][j] = haversine_km(locations[i][1], locations[i][2],
                                                       locations[j][1], locations[j][2]) * 1.25
                else:
                    # Fallback to haversine
                    dist_matrix[i][j] = haversine_km(locations[i][1], locations[i][2],
                                                   locations[j][1], locations[j][2]) * 1.25
        
        print("✅ Real road distances calculated!")
        
    else:
        print("🔄 Using haversine distances...")
        for i in range(n):
            for j in range(n):
                if i == j:
                    dist_matrix[i][j] = 0
                else:
                    dist_matrix[i][j] = haversine_km(locations[i][1], locations[i][2],
                                                   locations[j][1], locations[j][2]) * 1.25
    
    return dist_matrix

# Calculate distances
distance_matrix = calculate_distance_matrix(locations, G if network_success else None)

🛣️ Calculating real road distances...
   Warehouse: found node 2382893441
   Shop A: found node 12396712432
   Shop B: found node 632556609
   Shop C: found node 6438333515
   Shop D: found node 565894351
   Shop E: found node 7650416265
   Shop F: found node 2954611566
   Shop G: found node 5279037983
   Shop H: found node 6474635028
✅ Real road distances calculated!


In [22]:
# Display the distance matrix
print("\n📊 Distance Matrix (km):")
print("     ", end="")
for i in range(len(locations)):
    print(f"{i:6}", end="")
print()

for i in range(len(locations)):
    print(f"{i:2}: ", end="")
    for j in range(len(locations)):
        print(f"{distance_matrix[i][j]:6.1f}", end="")
    print(f"  ({locations[i][0]})")

print(f"\n🚛 Distance calculation method: {'OSMnx Real Roads' if network_success else 'Haversine + 25%'}")


📊 Distance Matrix (km):
          0     1     2     3     4     5     6     7     8
 0:    0.0   1.9   1.2   2.8   2.7   3.2   3.2   2.8   2.9  (Warehouse)
 1:    1.9   0.0   2.8   2.9   4.2   1.8   3.4   4.2   1.8  (Shop A)
 2:    1.2   2.8   0.0   2.4   3.2   4.4   4.3   1.6   3.1  (Shop B)
 3:    2.8   2.9   2.4   0.0   5.4   4.7   5.9   2.5   1.8  (Shop C)
 4:    2.7   4.2   3.2   5.4   0.0   4.7   2.6   4.3   5.6  (Shop D)
 5:    3.2   1.8   4.4   4.7   4.7   0.0   2.9   5.9   3.3  (Shop E)
 6:    3.2   3.4   4.3   5.9   2.6   2.9   0.0   5.8   5.2  (Shop F)
 7:    2.8   4.2   1.6   2.5   4.3   5.9   5.8   0.0   4.0  (Shop G)
 8:    2.9   1.8   3.1   1.8   5.6   3.3   5.2   4.0   0.0  (Shop H)

🚛 Distance calculation method: OSMnx Real Roads


In [1]:
# Step 5: Prepare the data for OR-Tools optimization
%pip install --upgrade protobuf

from ortools.constraint_solver import pywrapcp, routing_enums_pb2


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ortools 9.14.6206 requires protobuf<6.32,>=6.31.1, but you have protobuf 6.32.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [23]:
# Prepare for OR-Tools
SCALE = 100
dist_matrix_int = [[int(round(d * SCALE)) for d in row] for row in distance_matrix]

# Flatten vehicle specs
vehicle_specs = []
for v in vehicles:
    for _ in range(v["count"]):
        vehicle_specs.append((v["capacity"], v["cost_per_km"]))

print(f"\n🔧 OR-Tools VRP Setup:")
print(f"   Locations: {len(locations)}")
print(f"   Vehicles: {len(vehicle_specs)}")
for i, (cap, cost) in enumerate(vehicle_specs):
    print(f"     Vehicle {i}: {cap} bags, ₨{cost}/km")

# Create routing model
manager = pywrapcp.RoutingIndexManager(len(dist_matrix_int), len(vehicle_specs), 0)
routing = pywrapcp.RoutingModel(manager)
print("✅ Routing model created!")


🔧 OR-Tools VRP Setup:
   Locations: 9
   Vehicles: 4
     Vehicle 0: 200 bags, ₨100.0/km
     Vehicle 1: 80 bags, ₨70.0/km
     Vehicle 2: 80 bags, ₨70.0/km
     Vehicle 3: 50 bags, ₨50.0/km


NameError: name 'pywrapcp' is not defined